In [37]:
import pandas as pd
import numpy as np

In [38]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [39]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [40]:
from sklearn.model_selection import train_test_split

In [41]:
temp = df[['temp']]

In [42]:
from sklearn.preprocessing import MinMaxScaler
values = temp.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [43]:
scaled = scaled.reshape((19706,))

In [44]:
scaled.shape

(19706,)

In [45]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [46]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [52]:
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [53]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)                    │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.0250 - root_mean_squared_error: 0.1427 - val_loss: 0.0050 - val_root_mean_squared_error: 0.0709 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0048 - root_mean_squared_error: 0.0689 - val_loss: 0.0047 - val_root_mean_squared_error: 0.0689 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0046 - root_mean_squared_error: 0.0680 - val_loss: 0.0051 - val_root_mean_squared_error: 0.0717 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0046 - root_mean_squared_error: 0.0675 - val_loss: 0.0047 - val_root_mean_squared_error: 0.0682 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0045 - root_mean_squared_error: 0.0672 - val_loss: 0.0045 - val_root_mean_squared_error: 0.0673 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0044 - root_mean_squared_

In [54]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [55]:
poll = np.array(df["temp"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [56]:
from sklearn.metrics import mean_squared_error

In [57]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 0.55300534


In [58]:
forecast

array([[0.5388701 , 0.5238198 , 0.52352834, ..., 0.60559887, 0.5902555 ,
        0.57325995],
       [0.8818791 , 0.8673883 , 0.84816957, ..., 0.7438789 , 0.75232553,
        0.796959  ],
       [0.5157469 , 0.5943804 , 0.6199081 , ..., 0.43599945, 0.4717365 ,
        0.49238998],
       ...,
       [0.39043373, 0.3692348 , 0.3686052 , ..., 0.2906848 , 0.3165629 ,
        0.39325747],
       [0.5890337 , 0.61486745, 0.66718566, ..., 0.48431444, 0.47819823,
        0.4782009 ],
       [0.5541259 , 0.517329  , 0.5086943 , ..., 0.61185694, 0.5514315 ,
        0.5335064 ]], dtype=float32)

In [59]:
testPredict

array([[28.670357, 28.539068, 28.536526, ..., 29.25245 , 29.118607,
        28.970348],
       [31.662521, 31.536114, 31.368465, ..., 30.458706, 30.532389,
        30.92174 ],
       [28.468647, 29.154589, 29.377274, ..., 27.772987, 28.084732,
        28.264896],
       ...,
       [27.375504, 27.190578, 27.185087, ..., 26.505365, 26.731108,
        27.400135],
       [29.107948, 29.333303, 29.78969 , ..., 28.194452, 28.141098,
        28.14112 ],
       [28.803436, 28.482447, 28.407124, ..., 29.307041, 28.779934,
        28.623568]], dtype=float32)

In [60]:
y_test_true

array([[28.663692, 28.663692, 28.744278, ..., 29.630709, 29.630709,
        29.489685],
       [31.645325, 31.423717, 31.343132, ..., 31.947517, 32.08854 ,
        32.551903],
       [28.442085, 29.268078, 29.489685, ..., 28.280916, 29.026325,
        29.106909],
       ...,
       [27.394485, 27.172878, 27.172878, ..., 25.762648, 25.984255,
        26.508055],
       [28.804716, 28.965885, 29.409101, ..., 28.663692, 28.663692,
        28.663692],
       [28.804716, 28.220478, 28.220478, ..., 29.187494, 28.280916,
        28.139893]], dtype=float32)

In [62]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [63]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.1372
RMSE for segment 2: 0.1890
RMSE for segment 3: 0.2286
RMSE for segment 4: 0.2678
RMSE for segment 5: 0.2927
RMSE for segment 6: 0.3161
RMSE for segment 7: 0.3300
RMSE for segment 8: 0.3461
RMSE for segment 9: 0.3608
RMSE for segment 10: 0.3675
RMSE for segment 11: 0.3756
RMSE for segment 12: 0.3801
RMSE for segment 13: 0.3783
RMSE for segment 14: 0.3813
RMSE for segment 15: 0.3806
RMSE for segment 16: 0.3811
RMSE for segment 17: 0.3859
RMSE for segment 18: 0.3873
RMSE for segment 19: 0.3922
RMSE for segment 20: 0.3948
RMSE for segment 21: 0.3958
RMSE for segment 22: 0.3996
RMSE for segment 23: 0.3997
RMSE for segment 24: 0.4003
RMSE for segment 25: 0.4094
RMSE for segment 26: 0.4163
RMSE for segment 27: 0.4258
RMSE for segment 28: 0.4373
RMSE for segment 29: 0.4479
RMSE for segment 30: 0.4521
RMSE for segment 31: 0.4586
RMSE for segment 32: 0.4637
RMSE for segment 33: 0.4667
RMSE for segment 34: 0.4695
RMSE for segment 35: 0.4751
RMSE for segment 36: 0.4794
R

In [61]:
model.save("temp_1dcnn.keras")